In [1]:
import sqlite3
import pandas as pd
import numpy as np

# --- STEP 1: Create a Mock Financial Database ---
# This creates a database in-memory so you don't have to manage files.
conn = sqlite3.connect(':memory:')

# Create a dummy dataset (simulating 100 days of market data)
tickers = ['AAPL', 'GE', 'PSCT', 'MSFT', 'TSLA']
data = []
for t in tickers:
    for i in range(100):
        data.append([t, f'2023-01-{i+1:02}', 100 + np.random.randn(), 105 + np.random.randn(), 95 + np.random.randn(), 101 + np.random.randn()])

df = pd.DataFrame(data, columns=['ticker', 'date', 'open', 'high', 'low', 'close'])

# Load the dataframe into a SQL table called 'market_data'
df.to_sql('market_data', conn, index=False)

# --- STEP 2: The "Applied SQL" Queries ---
# These are the ones Linda's recruiter wants to see.

def run_query(query, title):
    print(f"\n--- {title} ---")
    display(pd.read_sql_query(query, conn))

# Query A: Find the Top 5 most volatile days (Business Logic)
query_vol = """
SELECT ticker, date, ((high - low) / open) * 100 AS vol_pct
FROM market_data
ORDER BY vol_pct DESC
LIMIT 5;
"""

# Query B: 5-Day Moving Average using a Window Function (Technical Mastery)
query_ma = """
SELECT ticker, date, close,
       AVG(close) OVER (PARTITION BY ticker ORDER BY date ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) as moving_avg
FROM market_data
WHERE ticker = 'AAPL'
LIMIT 10;
"""

run_query(query_vol, "Volatility Audit")
run_query(query_ma, "Window Function: Moving Average")


--- Volatility Audit ---


,ticker,date,vol_pct
0,GE,2023-01-11,14.023190
1,MSFT,2023-01-07,13.988091
2,PSCT,2023-01-85,13.639283
3,TSLA,2023-01-87,13.468318
4,GE,2023-01-32,13.412169



--- Window Function: Moving Average ---


,ticker,date,close,moving_avg
0,AAPL,2023-01-01,101.344082,101.344082
1,AAPL,2023-01-02,102.470640,101.907361
2,AAPL,2023-01-03,99.318289,101.044337
3,AAPL,2023-01-04,101.095676,101.057172
4,AAPL,2023-01-05,101.513596,101.148457
5,AAPL,2023-01-06,100.969749,101.073590
6,AAPL,2023-01-07,100.637302,100.706922
7,AAPL,2023-01-08,101.096398,101.062544
8,AAPL,2023-01-09,102.757744,101.394958
9,AAPL,2023-01-10,103.117548,101.715748


In [4]:
%%writefile app.py
import streamlit as st
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt

# 1. Setup Mock Database (SQL logic)
conn = sqlite3.connect(':memory:', check_same_thread=False)
tickers = ['AAPL', 'GE', 'PSCT', 'MSFT', 'TSLA', 'SPY', 'QQQ']
data = []
for t in tickers:
    for i in range(100):
        # Generating synthetic data with some "errors" for the audit to find
        base_price = 100 if t != 'TSLA' else 500
        vol = 2 if t != 'TSLA' else 15 # TSLA will be our "Volatility Outlier"
        data.append([t, f'2023-01-{i+1:02}', base_price, base_price + vol, base_price - vol, base_price + 1])

df = pd.DataFrame(data, columns=['ticker', 'date', 'open', 'high', 'low', 'close'])
df.to_sql('market_data', conn, index=False)

# 2. Dashboard UI
st.set_page_config(page_title="SQL Data Audit", layout="wide")
st.title("🔍 SQL Financial Data Audit Dashboard")
st.markdown("This dashboard runs real-time SQL queries to identify data outliers and integrity issues.")

# 3. SQL Query 1: Volatility Outliers
st.subheader("1. High Volatility Audit (SQL Query)")
query_vol = """
SELECT ticker, AVG(((high - low) / open) * 100) AS avg_daily_vol
FROM market_data
GROUP BY ticker
ORDER BY avg_daily_vol DESC
"""
vol_df = pd.read_sql_query(query_vol, conn)

col1, col2 = st.columns([1, 2])
with col1:
    st.write("Targeting assets with >5% average range.")
    st.dataframe(vol_df)
with col2:
    st.bar_chart(data=vol_df, x='ticker', y='avg_daily_vol')

# 4. SQL Query 2: Data Integrity (Sample Size Check)
st.subheader("2. Data Completeness Check")
query_integrity = """
SELECT ticker, COUNT(date) as data_points
FROM market_data
GROUP BY ticker
"""
integrity_df = pd.read_sql_query(query_integrity, conn)
st.table(integrity_df)

st.sidebar.info("This dashboard uses a SQLite in-memory database to simulate a production data environment.")

Overwriting app.py


In [5]:
!pip install -q streamlit
!npm install -q localtunnel
!streamlit run app.py & npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 3s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸⠙⠹your url is: https://eight-lies-relate.loca.lt



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.53.127.188:8501

  Stopping...
^C
